In [ ]:
import requests
import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import  KFold
import warnings
import re
warnings.filterwarnings('ignore')

In [ ]:
movies_db = pd.read_csv("movies_db_1.csv", delimiter=",",engine='python') #Provient de 1_Recuperation_Movies

In [ ]:
movies_db.dropna(inplace=True) #Nettoyage des Na

In [ ]:
movies_db['actors'] = movies_db['actors'].str.strip('[]')
movies_db['director'] = movies_db['director'].str.strip('[]')
movies_db['producers'] = movies_db['producers'].str.strip('[]')

In [ ]:
def extract_popularities(col):
  movies_db_col = movies_db[col].apply(lambda s : re.findall('\'popularity\': \d+.?\d*',s))
  return movies_db_col.apply(lambda s :  np.sum([float(re.search('(\'popularity\': )(\d+.?\d*)', x).group(2)) for x in s]))

In [ ]:
movies_actors_popularities = movies_db['actors'].apply(lambda s : re.findall('\'popularity\': \d+.?\d*',s))

In [ ]:
movies_actors_popularities = movies_actors_popularities.apply(lambda s :  np.sum([float(re.search('(\'popularity\': )(\d+.?\d*)', x).group(2)) for x in s]))

In [ ]:
movies_db['pop_actors'] = extract_popularities('actors')
movies_db['pop_directors'] = extract_popularities('director')
movies_db['pop_producer'] = extract_popularities('producers')

In [ ]:
convert_dict = {'budget':'int64',
                'revenue':'int64',
                'runtime':'float64',
                'vote_count':'int64'
                }

movies_db[['budget','revenue','runtime','vote_count']] = movies_db[['budget','revenue','runtime','vote_count']].astype(convert_dict)

In [ ]:
movies_db['release_date'] = pd.to_datetime(movies_db['release_date'])

In [ ]:
movies_db.drop(movies_db[movies_db['revenue']<=0].index,inplace=True)

In [ ]:
movies_db.drop(movies_db[movies_db['runtime']==0].index,inplace=True)

In [ ]:
movies_db["year"] = movies_db["release_date"].apply(lambda s: s.year)

In [ ]:
movies_db.drop(movies_db[movies_db["year"]==2023].index,inplace=True)

In [ ]:
movies_db['director'] = movies_db['director'].str.strip('[]')
movies_db['producers'] = movies_db['producers'].str.strip('[]')
movies_db['genres'] = movies_db['genres'].str.strip('[]')
movies_db['production_companies'] = movies_db['production_companies'].str.strip('[]')

In [ ]:
movies_db['actors'] = movies_db['actors'].apply(lambda s : re.findall('[a-zA-Z]+ [a-zA-Z]*',s))

In [ ]:
movies_db = movies_db[movies_db['year']>=2010]

In [ ]:
movies_db.reset_index(inplace=True)
movies_db

In [ ]:
movies_db.drop(["index"],axis=1,inplace=True)

Importation données Youtube

In [ ]:
youtube = pd.read_csv("Film_View_Like_BIS.csv",engine='python',names=["count_view","count_like"],sep=None)

In [ ]:
youtube.reset_index(inplace=True)

In [ ]:
youtube.rename(columns={"index":"title"},inplace=True)

In [ ]:
movies_db['title']=movies_db['title'].apply(lambda s: s.replace(":"," "))
movies_db['title']=movies_db['title'].apply(lambda s: s.replace("-"," "))
youtube['title']=youtube['title'].apply(lambda s: s.replace('"',''))
youtube['title']=youtube['title'].apply(lambda s: s.replace(":"," "))
youtube['title']=youtube['title'].apply(lambda s: s.replace("-"," "))

In [ ]:
movies_db=pd.merge(movies_db,youtube,on=["title"])

Export CSV

In [ ]:
movies_db.to_csv("movies_db_2.csv",index=False)